Deriving consensus annotations and analyzing agreement within observer groups

Analytic code supporting "Observer variability in manual-visual interpretation of aerial imagery of wildlife, with implications for deep learning" - Converse et al. submitted Feb 2024

In [ ]:
#Imports
import pandas as pd
import numpy as np
import ast
import sklearn.metrics
from shapely.geometry import Polygon,Point
import matplotlib.pyplot as plt
import shapely
import cv2 as cv
import os
import gc

#Data loading
path = "path/to/labels.csv"
with open(path) as f:
    raw = pd.read_csv(f)

In [ ]:
#Bounding box area; area by class:

#Calculate area of bounding boxes
def calc_area(row):
    bbox = row['bbox']
    xmin, ymin, w, h = bbox
    return w * h

raw['area'] = raw.apply(calc_area, axis=1)

#Determine average area of bounding box per class
raw.groupby("class_id")["area"].mean()

Derive consensus bounding boxes with DBSCAN

In [ ]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import patches, text, patheffects
import cv2
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
import scipy.cluster.hierarchy as hcluster
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import scipy.stats

#Define parameters for DBSCAN

def group_DBSCAN(df):
    x = df[["c_x", "c_y"]].to_numpy()
    cluster = DBSCAN(eps=15, min_samples=5).fit(x)
    labels = cluster.labels_
    df["cluster_id"] = labels
    return labels

#Derive bounding box centers as inputs for DBSCAN
bboxes = raw["bbox"]
c_x = []
c_y = []
x = []
y = []
w = []
h = []
centers = []
for coord in bboxes:
    center = (coord[0]+(coord[2]/2), coord[1]+(coord[3]/2))
    c_x.append(center[0])
    c_y.append(center[1])
    x.append(coord[0])
    y.append(coord[1])
    w.append(coord[2])
    h.append(coord[3])
    centers.append(center)
#Make these centers into a coordinate format
coords = []
for row in centers:
    coord = list(row)
    coords.append(coord)
#Append new columns to dataframe
raw["c_x"] = c_x
raw["c_y"] = c_y
raw['x'] = x
raw['y'] = y
raw['w'] = w
raw['h'] = h

In [ ]:
#Identify clusters of labels with DBSCAN; append cluster ID to individual labels

clusters = raw.groupby("filename").apply(lambda x: group_DBSCAN(x))
clusters = clusters.reset_index()
clusters.rename(columns = {0:'cluster_id'}, inplace=True)
long = clusters.explode("cluster_id")
long.reset_index()
filesort = raw.sort_values(["filename", "annotation_id"])
filesort.reset_index()
test = filesort.reset_index().merge(long.reset_index(), left_index=True, right_index=True, how='left')
test = test.drop(columns=['filename_y','index_y'])
test = test.rename(columns={'filename_x':'filename', 'index_x': 'index'})

In [ ]:
#Derive consensus labels

#Dictionary of category values
categories = {1: "Crane", 2: 'Goose', 3:'Duck', 4: 'Other Bird'}

#Derive refined products: median coordinates, and plurality vote class ID
refined = test.groupby(['filename', 'cluster_id']).agg({'x':'median', 
                         'y':'median', 
                         'w':'median', 
                         'h': 'median',
                         'category_id': pd.Series.mode}).reset_index()
#Make median bounding box into its own column in list form
refined['bbox']= refined[['x','y','w','h']].values.tolist()
#Remove rows where no agreement was reached on class ID
rowcount = len(refined)
refined_drop = refined[refined['category_id'].isin([1,2,3,4])]
agreement = len(refined_drop)
removed = rowcount - agreement
#Remove noise points
refined_id = refined_drop[refined_drop['cluster_id'] != -1]
final = len(refined_id)
noise = rowcount - final
print("Removed rows: %s due to no class ID agreement; %s noise points" %(removed, noise))
refined_id['category'] = refined_id["category_id"].map(categories)
refined_id = refined_id.drop(columns=['x','y','w','h'])

#Create analysis dataframe that has raw annotations with the corresponding consensus annotation
df = test.merge(refined_id, left_on=['filename','cluster_id'], right_on = ['filename','cluster_id'], how='left')
df = df.drop(columns=['c_x','c_y','area','x','y','w','h'])
df = df.rename(columns={'bbox_x': 'bbox_orig', 'category_id_x': 'cat_id_orig', "category_x": "cat_orig", 'bbox_y': 'bbox_refined', 'category_id_y': 'cat_id_refined', "category_y": "cat_refined"})

In [ ]:
#Counting number of dropped annotations that were not matched with a cluster
missing = df[df["cluster_id"] == -1]
len(missing)

ID and Locational Agreement between raw and consensus labels

In [ ]:
#Calculate agreement between each raw label ID and the corresponding consensus label ID
df['agree'] = 0
df.loc[df['cat_orig'] == df["cat_refined"], 'agree'] = 1

#Calculate statistics on agreement overall and per class
grouped_data = df.groupby("cat_refined")["agree"].agg(["mean", "std"]).reset_index()
print(df['agree'].mean())
print(df['agree'].std())

In [ ]:
#Calculating IOU between each raw bounding box and the corresponding consensus box
from shapely.geometry import box

def eval_bbox(row, col_name):
    bbox_str = row[col_name]
    if pd.notnull(bbox_str):
        bbox = np.array(ast.literal_eval(bbox_str))
        bbox = bbox.astype(float)
    else:
        bbox = np.array([np.nan, np.nan, np.nan, np.nan])
    return bbox

# Define a function to calculate the IOU only if both bounding boxes are non-null
def calculate_iou(row):
    bbox_orig = eval_bbox(row, 'bbox_orig')
    bbox_ref = eval_bbox(row, 'bbox_refined')
    if np.isnan(bbox_orig[0]) or np.isnan(bbox_orig[1]) or np.isnan(bbox_orig[2]) or np.isnan(bbox_orig[3]) or \
        np.isnan(bbox_ref[0]) or np.isnan(bbox_ref[1]) or np.isnan(bbox_ref[2]) or np.isnan(bbox_ref[3]):
        iou = None
    else:
        bbox_orig = box(bbox_orig[0], bbox_orig[1], bbox_orig[0] + bbox_orig[2], bbox_orig[1] + bbox_orig[3])
        bbox_ref = box(bbox_ref[0], bbox_ref[1], bbox_ref[0] + bbox_ref[2], bbox_ref[1] + bbox_ref[3])
        iou = bbox_orig.intersection(bbox_ref).area / bbox_orig.union(bbox_ref).area
    return iou

# Apply the function to each row of the DataFrame and save the results in a new column
df['IOU'] = df.apply(calculate_iou, axis=1)

#IOU statistics overall and per class
df["IOU"].mean()
df.groupby("cat_refined")["IOU"].mean()

In [ ]:
#CALCULATING PIELOU'S INDEX

# Group the dataframe by image, then by cluster
grouped = df.groupby(['filename', 'cluster_id'])

# Create empty lists to store the results
cluster_id_list = []
filename_list = []
consensus_class_id_list = []
class_count_list = []
bbox_list = []
pielou_index_list = []

# Loop through each group and calculate Pielou's evenness index
for name, group in grouped:
    # Get the cluster ID, filename, and consensus class ID for this group
    cluster_id = name[1]
    filename = name[0]
    #ADJUST LINE BELOW FOR SPP VS SUPERCLASS
    consensus_bbox = group['bbox_refined'].iloc[0] 
    consensus_class_id = group['cat_refined'].iloc[0]  # Assumes all consensus IDs in the group are the same
    
    # Count the number of annotations in the group
    num_annotations = len(group)
    
    # Count the number of annotations for each original class ID (ADJUST HERE FOR SPP VS SUPERCLASS)
    class_counts = group.groupby('orig_superclass').size().values

    #Score complete agreement as zero; otherwise, calculate Pielou:
    if len(class_counts) == 1:
        evenness_index = 0
    else:
        # Calculate the relative abundance of each original class ID
        relative_abundance = class_counts / num_annotations
    
        # Calculate the evenness index using Pielou's formula
        evenness_index = -np.sum(relative_abundance * np.log(relative_abundance)) / np.log(len(relative_abundance))
    
    # Append the results to the lists
    cluster_id_list.append(cluster_id)
    filename_list.append(filename)
    consensus_class_id_list.append(consensus_class_id)
    #agreement = class_count_list.append(class_counts)
    bbox_list.append(consensus_bbox)
    pielou_index_list.append(evenness_index)

# Create a new dataframe with the results
pielou = pd.DataFrame({
    'cluster_id': cluster_id_list,
    'filename': filename_list,
    'consensus_class_ID': consensus_class_id_list,
    'bbox': bbox_list,
    'pielou_index': pielou_index_list
})

#Pielou Index statistics overall and per class
print(pielou["pielou_index"].mean())
print(pielou["pielou_index"].std())
print(pielou.groupby("consensus_class_ID")["pielou_index"].mean())
print(pielou.groupby("consensus_class_ID")["pielou_index"].std())